In [8]:
import torch
import torch.nn as nn

In [9]:
class Dense(nn.Module):
    def __init__(self, input_size, output_size):
        super(Dense, self).__init__()
        uniform_range = 1.0 / input_size
        uniform_distribution = torch.distributions.uniform.Uniform(-uniform_range, uniform_range)
        self.Weights = nn.Parameter(uniform_distribution.sample((input_size, output_size)))
        self.Bias = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        return torch.matmul(x, self.Weights) + self.Bias
    
    def __str__(self):
        return f'''
Dense Layer
input size : {self.Weights.size(0)}, output size : {self.Weights.size(1)}
Weights :
{self.Weights}
Bias :
{self.Bias}
'''

In [10]:
class NNUE_V1(nn.Module):
    def __init__(self):
        super(NNUE_V1, self).__init__()

        self.ProcessPlayer = Dense(41024, 256)

        self.FirstDense = Dense(512, 32)
        self.SecondDense = Dense(32, 32)
        self.Output = Dense(32, 1)

        self.Activation = torch.relu
    
    def forward(self, x):
        # x dimentions : batch x 2 x 41024
        FirstToPlay  = self.Activation(self.ProcessPlayer(x[:, 0, :])) # batch x 256
        SecondToPlay = self.Activation(self.ProcessPlayer(x[:, 1, :])) # batch x 256

        x = torch.cat((FirstToPlay, SecondToPlay), dim=1)
        
        x = self.Activation(self.FirstDense(x))
        x = self.Activation(self.SecondDense(x))
        x = self.Output(x)

        return x

In [11]:
def decode(encoded_str: str):
    parts = encoded_str.split()
    evaluation = int(parts[-1])
    tensor = torch.zeros((6, 8, 8), dtype=torch.int)

    def decode_layer(encoded_value, sign):
        decoded_layer = torch.zeros((8, 8), dtype=torch.int)
        for j in range(64):
            if encoded_value & (1 << (63 - j)):
                decoded_layer[j // 8, j % 8] = sign
        return decoded_layer

    for i in range(6):
        for sign in [-1, 1]:
            encoded_value = int(parts[2 * i + (sign == -1)])
            tensor[i] += decode_layer(encoded_value, sign)

    return tensor, evaluation

def decode_2(encoded_str: str):
    ret = torch.zeros(2, 41024)

    tnsr, eval = decode(encoded_str)

    # First to move
    King_Type_Cord = torch.zeros(64, 10, 64)

    # Find King
    king1 = -1
    for i in range(64):
        if tnsr[5, i // 8, i % 8] == 1:
            king1 = i
            break
    
    # For every piece square
    for i in range(64):
        for j in range(10):
            sign = 0
            if j % 2 == 0 :
                sign = 1
            else:
                sign = -1
            idx = (j+2)//2
            if tnsr[idx, i // 8, i % 8] == sign:
                King_Type_Cord[king1, j, i] = 1
    
    # Find enemy king
    king2 = -1
    for i in range(64):
        if tnsr[5, i // 8, i % 8] == -1:
            king2 = i
            break
    
    Additional = torch.zeros(64)
    Additional[king2] = 1

    ret[0] = torch.cat((King_Type_Cord.view(64*10*64), Additional.view(64)))

    # Second to move
    King_Type_Cord = torch.zeros(64, 10, 64)
    # For every piece square
    for i in range(64):
        for j in range(10):
            sign = 0
            if j % 2 == 0 :
                sign = -1
            else:
                sign = 1
            idx = (j+2)//2
            if tnsr[idx, i // 8, i % 8] == sign:
                King_Type_Cord[king1, j, i] = 1

    Additional = torch.zeros(64)
    Additional[king1] = 1
    
    ret[1] = torch.cat((King_Type_Cord.view(64*10*64), Additional.view(64)))

    return ret, torch.tensor([eval])
    
class Dataset(torch.utils.data.Dataset):
    def __init__(self, file_path):
        self.path = file_path
        self.file = open(file_path)
        self.lines = self.file.readlines()
    
    def __len__(self):
        return len(self.lines)
    
    def __getitem__(self, idx):
        # output : 2 x 41024
        line = self.lines[idx]
        tnr, eval = decode_2(line)
        return tnr, eval

In [12]:
import chess
import torch

def board_to_tensor(board):
    tensor = torch.zeros(6, 8, 8)
    for file in range(8):
        for rank in range(8):
            square = chess.square(file, rank)
            piece = board.piece_at(square)
            if piece:
                piecet = piece.piece_type
                number = 1 if piece.color == chess.WHITE else -1
                tensor[int(piecet)-1, file, rank] = number
    return tensor

INIT_BOARD = board_to_tensor(chess.Board())

class TensorBoard:
    def __init__(self):
        self.board = chess.Board()
        self.tensor = INIT_BOARD.clone()

    def push(self, move: chess.Move):
        # Get the piece type and color
        from_square = move.from_square
        to_square = move.to_square
        piece = self.board.piece_at(from_square)
        piecet = piece.piece_type
        color = 1 if piece.color == chess.WHITE else -1

        # Clear the 'from' square
        from_file, from_rank = chess.square_file(from_square), chess.square_rank(from_square)
        self.tensor[:, from_file, from_rank] = 0

        # Set the 'to' square
        to_file, to_rank = chess.square_file(to_square), chess.square_rank(to_square)
        self.tensor[:, to_file, to_rank] = 0  # Clear any captured piece
        self.tensor[int(piecet)-1, to_file, to_rank] = color

        # Handle special moves
        if self.board.is_castling(move):
            if to_file > from_file:  # Kingside castling
                rook_from, rook_to = chess.H1 if color == 1 else chess.H8, chess.F1 if color == 1 else chess.F8
            else:  # Queenside castling
                rook_from, rook_to = chess.A1 if color == 1 else chess.A8, chess.D1 if color == 1 else chess.D8
            rook_from_file, rook_from_rank = chess.square_file(rook_from), chess.square_rank(rook_from)
            rook_to_file, rook_to_rank = chess.square_file(rook_to), chess.square_rank(rook_to)
            self.tensor[int(chess.ROOK)-1, rook_from_file, rook_from_rank] = 0
            self.tensor[int(chess.ROOK)-1, rook_to_file, rook_to_rank] = color
        elif self.board.is_en_passant(move):
            captured_pawn = chess.square(to_file, from_rank)
            captured_file, captured_rank = chess.square_file(captured_pawn), chess.square_rank(captured_pawn)
            self.tensor[int(chess.PAWN)-1, captured_file, captured_rank] = 0
        elif move.promotion:
            self.tensor[int(piecet)-1, to_file, to_rank] = 0
            self.tensor[int(move.promotion)-1, to_file, to_rank] = color

        # Update the underlying chess board
        self.board.push(move)


    def pop(self):
        move = self.board.pop()
        
        # Reverse the move
        from_square = move.to_square
        to_square = move.from_square
        piece = self.board.piece_at(to_square)
        piecet = piece.piece_type
        color = 1 if piece.color == chess.WHITE else -1

        # Clear the 'from' square (which was the 'to' square in the original move)
        from_file, from_rank = chess.square_file(from_square), chess.square_rank(from_square)
        self.tensor[:, from_file, from_rank] = 0

        # Set the 'to' square (which was the 'from' square in the original move)
        to_file, to_rank = chess.square_file(to_square), chess.square_rank(to_square)
        self.tensor[int(piecet)-1, to_file, to_rank] = color

        # Handle special moves
        if self.board.is_castling(move):
            if from_file > to_file:  # Kingside castling
                rook_to, rook_from = chess.H1 if color == 1 else chess.H8, chess.F1 if color == 1 else chess.F8
            else:  # Queenside castling
                rook_to, rook_from = chess.A1 if color == 1 else chess.A8, chess.D1 if color == 1 else chess.D8
            rook_to_file, rook_to_rank = chess.square_file(rook_to), chess.square_rank(rook_to)
            rook_from_file, rook_from_rank = chess.square_file(rook_from), chess.square_rank(rook_from)
            self.tensor[int(chess.ROOK)-1, rook_to_file, rook_to_rank] = 0
            self.tensor[int(chess.ROOK)-1, rook_from_file, rook_from_rank] = color
        elif self.board.is_en_passant(move):
            captured_pawn = chess.square(from_file, to_rank)
            captured_file, captured_rank = chess.square_file(captured_pawn), chess.square_rank(captured_pawn)
            self.tensor[int(chess.PAWN)-1, captured_file, captured_rank] = -color
        elif move.promotion:
            self.tensor[int(move.promotion)-1, from_file, from_rank] = 0
            self.tensor[int(chess.PAWN)-1, to_file, to_rank] = color

        # If a piece was captured, restore it
        captured_piece = self.board.piece_at(from_square)
        if captured_piece:
            captured_piecet = captured_piece.piece_type
            captured_color = 1 if captured_piece.color == chess.WHITE else -1
            self.tensor[int(captured_piecet)-1, from_file, from_rank] = captured_color

        return move

In [13]:
def Encode(tb : TensorBoard, evaluation : float):
    encoded = ''
    tnr = tb.tensor
    if tb.board.turn == chess.BLACK:
        tnr = -tnr
        evaluation = -evaluation
    for i in range(6):
        for sign in [-1, 1]:
            int_ = 0
            for j in range(64):
                if tnr[i, j//8, j%8] == sign:
                    int_ = 2*int_ + 1
                else:
                    int_ = 2*int_
            encoded += str(int_) + ' '
    
    return encoded+str(int(100*evaluation))

In [14]:
TB = TensorBoard()
TB.push(chess.Move.from_uci('e2e4'))
TB.push(chess.Move.from_uci('e7e5'))

print(Encode(TB, 0.0))

144680345776816642 4629771060831600704 281474976710912 36028797018996736 1099511693312 140737496743936 72057594037927937 9223372036854775936 4294967296 549755813888 16777216 2147483648 0


### Train the model

In [15]:
# Train the model
def Train(epochs, Dataloader_Train, Dataloader_Test, model, criterion, optimizer, device):
    lowest_loss = float('inf')
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        model.train()
        train_loss = 0.0

        for i, (tnr, eval) in enumerate(Dataloader_Train):
            tnr, eval = tnr.to(device), eval.to(device)
            optimizer.zero_grad()
            output = model(tnr)
            loss = criterion(output, eval)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * tnr.size(0)
        
        train_loss /= len(Dataloader_Train.dataset)

        model.eval()
        test_loss = 0.0
        with torch.inference_mode():
            for i, (tnr, eval) in enumerate(Dataloader_Test):
                tnr, eval = tnr.to(device), eval.to(device)
                output = model(tnr)
                loss = criterion(output, eval)
                test_loss += loss.item() * tnr.size(0)
        
        test_loss /= len(Dataloader_Test.dataset)

        print(f'Epoch : {epoch+1}/{epochs} Train Loss : {train_loss}, Test Loss : {test_loss}')

        if test_loss < lowest_loss:
            lowest_loss = test_loss
            torch.save(model.state_dict(), '../Model/NNUE_V1_best.pt')
            print('Model saved')
        
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = NNUE_V1().to(device)

    num_gpu = torch.cuda.device_count()
    if num_gpu > 1:
        model = nn.DataParallel(model)
    
    print(f'Training on {device}')
    print(f'Number of parameters : {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

    # Load the model
    # model.load_state_dict(torch.load('model.pth'))

    # Load the dataset
    dataset = Dataset('../Dataset/Processed/Dataset_V1.txt')

    # Divide the dataset into training set and validation set
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    # Define the dataloaders
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    Train(100, train_dataloader, test_dataloader, model, criterion, optimizer, device)

Training on cuda
Number of parameters : 10519905
Epoch 1/100


TypeError: cannot pickle 'TextIOWrapper' instances